# Notebook 5: The "Value at Risk" Dashboard.

What this Dashboard does:

* Inputs: You adjust the Uranium Price (Cost), Electricity Price (Revenue), and WACC (Risk).

* Engine: It instantly recalculates the EBITDA and NPV for every single reactor in the world (over 12,000 data points) based on your inputs.

* Output: It displays the Total Industry Valuation and plots the future cash flow profile.

Block 1: Setup & Data Preparation

We import the necessary interactive libraries and load our finalized valuation model. We also pre-calculate the "Non-Fuel" costs (O&M) since these don't change with the sliders, making the dashboard faster.

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set visualization style
sns.set_theme(style="whitegrid")

print("--- Notebook 5: The 'Value at Risk' Dashboard ---")

# 1. Load the Final Valuation Model
try:
    df_model = pd.read_csv('fleet_valuation_final.csv')
    print("Valuation Model loaded successfully.")
except FileNotFoundError:
    print("Error: 'fleet_valuation_final.csv' not found. Please run Notebook 4 first.")

# 2. Pre-Calculate Static Components to Speed Up Dashboard
# The Fixed and Variable O&M don't change when we move the Price sliders.
# We calculate them once here.
if 'Total_OM_Cost' not in df_model.columns:
    # Just in case the column wasn't saved, we reconstruct it
    df_model['Total_OM_Cost'] = df_model['Annual_Fixed_OM'] + df_model['Annual_Variable_OM']

print(f"Model ready with {len(df_model)} projection records.")

--- Notebook 5: The 'Value at Risk' Dashboard ---
Valuation Model loaded successfully.
Model ready with 11966 projection records.


Block 2: The Valuation Engine (Function)

This is the brain of the dashboard. It accepts the three variables from the sliders and performs the full DCF calculation on the fly.

In [17]:
def calculate_scenario(u_price, p_price, wacc):
    """
    Re-calculates the entire fleet valuation based on dynamic inputs.
    """
    # Constants
    CURRENT_YEAR = 2025
    U3O8_USAGE = 0.5 # lbs/MWh
    FIXED_FUEL_CYCLE = 4.0 # $/MWh

    # 1. Calculate Dynamic Revenue
    revenue = df_model['Annual_Generation_MWh'] * p_price

    # 2. Calculate Dynamic Fuel Cost
    # Cost = Generation * ( (U_Price * Usage) + Conversion/Enrichment )
    fuel_cost = df_model['Annual_Generation_MWh'] * ((u_price * U3O8_USAGE) + FIXED_FUEL_CYCLE)

    # 3. Calculate EBITDA
    # Revenue - (Fixed OM + Variable OM + Fuel Cost)
    # Note: 'Total_OM_Cost' is pre-calculated in Block 1
    total_cost = df_model['Total_OM_Cost'] + fuel_cost
    ebitda = revenue - total_cost

    # 4. Discounting (PV)
    # PV = EBITDA / (1 + WACC)^t
    years_from_start = df_model['Projection_Year'] - CURRENT_YEAR
    discount_factors = 1 / ((1 + wacc) ** years_from_start)
    pv_ebitda = ebitda * discount_factors

    # 5. Aggregate Results
    total_npv = pv_ebitda.sum()

    # Create a small aggregated dataframe for plotting
    df_plot = pd.DataFrame({
        'Year': df_model['Projection_Year'],
        'EBITDA': ebitda
    })
    df_plot = df_plot.groupby('Year')['EBITDA'].sum().reset_index()

    return total_npv, df_plot

Block 3: Building the Dashboard Interface

Here we define the sliders and the layout using ipywidgets.

In [18]:
# --- Widget Definitions ---

style = {'description_width': 'initial'}
layout = widgets.Layout(width='400px')

# 1. Uranium Price Slider ($/lb)
slider_uranium = widgets.FloatSlider(
    value=85, min=30, max=200, step=1,
    description='Uranium Price ($/lb U3O8)',
    style=style, layout=layout, continuous_update=False
)

# 2. Electricity Price Slider ($/MWh)
slider_power = widgets.FloatSlider(
    value=75, min=30, max=150, step=1,
    description='Electricity Price ($/MWh)',
    style=style, layout=layout, continuous_update=False
)

# 3. WACC Slider (%)
slider_wacc = widgets.FloatSlider(
    value=0.08, min=0.03, max=0.15, step=0.005,
    description='Discount Rate (WACC)',
    readout_format='.1%',
    style=style, layout=layout, continuous_update=False
)

# Output Area
out = widgets.Output()

Block 4: The Display Logic & Launch

This block connects the sliders to the calculation engine and updates the charts. Run this block to see the final dashboard!

In [19]:
def update_dashboard(change=None):
    # Get values from sliders
    u_price = slider_uranium.value
    p_price = slider_power.value
    wacc = slider_wacc.value

    # Run Calculation
    npv_result, df_chart = calculate_scenario(u_price, p_price, wacc)

    # Convert to Billions
    npv_billions = npv_result / 1e9
    df_chart['EBITDA_B'] = df_chart['EBITDA'] / 1e9

    with out:
        clear_output(wait=True)

        # --- Dashboard Header ---
        print(f"GLOBAL NUCLEAR FLEET VALUATION: ${npv_billions:,.2f} Billion")
        print("-" * 50)

        # --- Visualization ---
        plt.figure(figsize=(10, 5))

        # Plot EBITDA Profile
        plt.plot(df_chart['Year'], df_chart['EBITDA_B'], color='navy', linewidth=2, marker='o', markersize=4, label='Projected EBITDA')
        plt.fill_between(df_chart['Year'], df_chart['EBITDA_B'], 0, color='skyblue', alpha=0.3)

        plt.axhline(0, color='red', linewidth=1, linestyle='--')
        plt.title(f'Projected Industry Cash Flow\n(Inputs: ${p_price}/MWh | ${u_price}/lb U3O8)', fontsize=14)
        plt.ylabel('Annual EBITDA (Billions USD)')
        plt.xlabel('Year')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()

# Bind the update function to the sliders
slider_uranium.observe(update_dashboard, names='value')
slider_power.observe(update_dashboard, names='value')
slider_wacc.observe(update_dashboard, names='value')

# Initial Display
display(widgets.VBox([
    widgets.HTML("<h2>☢️ Nuclear Fleet Value-at-Risk Dashboard</h2>"),
    slider_power,
    slider_uranium,
    slider_wacc,
    out
]))

# Trigger initial load
update_dashboard()